# Tutorial - LangChain Chains

### Installing Required Libraries

In [21]:
#!pip install openai
#!pip install langchain
#!pip install python-dotenv
pip install IPython -q

SyntaxError: invalid syntax (2326706397.py, line 4)

### Load Environment Variables

In [6]:
def setup_openai():
    import os
    from openai import OpenAI
    from dotenv import load_dotenv, find_dotenv

    load_dotenv(find_dotenv(), override=True)
    api_key = os.environ.get('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
    print("API Key Loaded:", api_key is not None)
    return client

### Create OpenAI Client

In [7]:
client = setup_openai()

API Key Loaded: True


### Simple Chain Example

In [10]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()

template="""
You are an experienced Data Scientist with over 20 years of experience.
Write a few sentences about the following topic "{topic}" in {language}.
"""

prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

input_variables = {'topic':'Automatic Speech Recognition','language':'English'}

output = chain.invoke(input_variables)



> Entering new LLMChain chain...
Prompt after formatting:

You are an experienced Data Scientist with over 20 years of experience.
Write a few sentences about the following topic "Automatic Speech Recognition" in English.


> Finished chain.


In [11]:
print(output)

{'topic': 'Automatic Speech Recognition', 'language': 'English', 'text': 'Automatic Speech Recognition (ASR) is a technology that allows computers to transcribe spoken language into text. It utilizes advanced algorithms and machine learning techniques to accurately interpret and convert speech to text in real-time. ASR has a wide range of applications, from voice-controlled virtual assistants like Siri and Alexa to automated transcription services for meetings and interviews. As technology continues to advance, ASR systems are becoming increasingly accurate and reliable, revolutionizing the way we interact with and use spoken language in various industries.'}


In [12]:
template = 'What is the capital of {country}. List the top 3 places to visit in that country. Use bullet points'
prompt_tempate=PromptTemplate.from_template(template=template)
chain=LLMChain(
    llm=llm,
    prompt=prompt_tempate,
    verbose=True
)

country=input('Enter your country: ')
output=chain.invoke(country)
print(output)

Enter your country:  United Kingdom




> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of United Kingdom. List the top 3 places to visit in that country. Use bullet points

> Finished chain.
{'country': 'United Kingdom', 'text': 'The capital of the United Kingdom is London.\n\nTop 3 places to visit in the United Kingdom:\n- Buckingham Palace\n- The British Museum\n- The Tower of London'}


In [13]:
print(output['text'])

The capital of the United Kingdom is London.

Top 3 places to visit in the United Kingdom:
- Buckingham Palace
- The British Museum
- The Tower of London


### Sequential Chain Example

In [19]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# First Chain
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

prompt_template1 = PromptTemplate.from_template(
    template='''
    You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.
    '''
)

chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Second Chain
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

prompt_template2=PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)

chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Create the sequential chain
overall_chain = SimpleSequentialChain(chains=[chain1,chain2],verbose=True)
output=overall_chain.invoke('Binary Classification')
print(output['output'])



> Entering new SimpleSequentialChain chain...
Sure, here is an example of a function that implements binary classification using Python:

```python
def binary_classification(data, threshold):
    predictions = []
    for value in data:
        if value >= threshold:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

# Example usage
data = [0.1, 0.5, 0.8, 0.3, 0.6]
threshold = 0.5
predictions = binary_classification(data, threshold)
print(predictions)
```

In this function, we take a list of data points and a threshold value as input. For each data point, if the value is greater than or equal to the threshold, we predict a class label of 1, otherwise we predict a class label of 0. The function returns a list of these predictions.

This is a simple example of binary classification, where we are dividing the data into two classes based on a threshold value. In practice, more advanced machine learning algorithms like logistic regress

### 3-Step AI Storyteller: Prompt Generation, Evaluation, and Setting Creation

In [30]:
# Import necessary modules
from langchain.chains import SequentialChain, LLMChain 
# Import classes for creating chains of LLMs and executing them sequentially
from langchain_openai import ChatOpenAI 
# Import the ChatOpenAI class, which provides a wrapper for interacting with the OpenAI's chat models (like GPT-3.5 and GPT-4)
from langchain.prompts import PromptTemplate, ChatPromptTemplate 
# Import classes for defining the structure of prompts sent to the language models.
from IPython.display import HTML 
# Import the HTML class to display the results in a nicely formatted way within a Jupyter notebook

# Initialize the language model (You can experiment with different models)
llm = ChatOpenAI(temperature=1.0)  # Higher temperature for more creative output

# ------------------ Chain 1: Generate a Writing Prompt ------------------

# Create a ChatPromptTemplate to structure the prompt as a conversation
generate_prompt_template = ChatPromptTemplate.from_messages([
    ("human", "Generate a creative writing prompt that includes a genre, a character, and a conflict.")  # Instruction for the language model
])

# Create an LLMChain that uses the ChatOpenAI model and the prompt template
generate_prompt_chain = LLMChain(
    llm=llm,  
    prompt=generate_prompt_template,
    output_key="prompt"  # Name the output of this chain as "prompt" 
)

# ------------------ Chain 2: Evaluate the Prompt ------------------

# Create a PromptTemplate to evaluate the creativity of the generated prompt
evaluate_prompt_template = """Evaluate the following writing prompt on a scale of 1 to 5 
(1 being not creative and 5 being highly creative):

Prompt: {prompt}

Score:"""  

# Create an LLMChain to evaluate the prompt
evaluate_prompt_chain = LLMChain(
    llm=ChatOpenAI(temperature=0.0),  # Lower temperature for more objective evaluation
    prompt=evaluate_prompt,
    output_key="score"  # Name the output of this chain as "score"
)

# ------------------ Chain 3: Suggest a Setting ------------------

# Create a PromptTemplate to suggest a setting based on the generated prompt
suggest_setting_template = """Suggest an interesting setting for the following writing prompt:

Prompt: {prompt}

Setting:"""

suggest_setting_prompt = PromptTemplate(input_variables=["prompt"], template=suggest_setting_template)

# Create an LLMChain to suggest a setting
suggest_setting_chain = LLMChain(
    llm=llm, 
    prompt=suggest_setting_prompt,
    output_key="setting"  # Name the output of this chain as "setting"
)

# ------------------ Combine Chains into a SequentialChain ------------------

# Combine all three chains into a single sequential chain
overall_chain = SequentialChain(
    chains=[generate_prompt_chain, evaluate_prompt_chain, suggest_setting_chain],
    input_variables=[],   # This chain doesn't require any initial input
    output_variables=["prompt", "score", "setting"],  # Define the output keys we want
    verbose=True  # Optional: Show the chain's thought process for debugging
)

# ------------------ Run the Chain ------------------
# Execute the chain with an empty dictionary as input
result = overall_chain.invoke({})  
print("Generated Prompt:", result['prompt'])
print("Creativity Score:", result['score'])
print("Suggested Setting:", result['setting'])


# ------------------ Format and Display the Output ------------------
# Create an HTML string to format the results
html_output = f"""
<div style="border: 1px solid #ccc; padding: 20px; border-radius: 5px;">
  <h2>AI-Generated Writing Prompt</h2>
  <p>{result['prompt']}</p>
  <h3>Creativity Score</h3>
  <p>{result['score']}/5</p>
  <h3>Suggested Setting</h3>
  <p>{result['setting']}</p>
</div>
"""

# Display the formatted HTML in the Jupyter Notebook
HTML(html_output)


NameError: name 'evaluate_prompt' is not defined